<a href="https://colab.research.google.com/github/kyj098707/Deep-Learning-Paeper-Review-and-Code/blob/master/VGG.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
import torchvision
import torch.nn as nn
import torchvision.transforms as transforms

from tqdm.auto import tqdm

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
transform = transforms.Compose([
    transforms.Resize(256),
    transforms.RandomCrop(224),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
])

trainset = torchvision.datasets.STL10(root='./data', split='train', download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=32, shuffle=True)

testset = torchvision.datasets.STL10(root='./data', split='test', download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=32, shuffle=False)

  0%|          | 0/2640397119 [00:00<?, ?it/s]

Extracting ./data/stl10_binary.tar.gz to ./data
Files already downloaded and verified


In [ ]:
class VGG_11(nn.Module):
    def __init__(self, num_classes = 1000, init_weights = True):
        super(VGG_11,self).__init__()
        self.conv1 = nn.Sequential(
            nn.Conv2d(in_channels=3, out_channels=64,kernel_size=3, padding = 1,stride=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2)
            )
        self.conv2 = nn.Sequential(
            nn.Conv2d(in_channels=64, out_channels=128,kernel_size=3, padding = 1,stride=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2)
            )
        
        self.conv3 = nn.Sequential(
            nn.Conv2d(in_channels=128, out_channels=256,kernel_size=3, padding = 1,stride=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(in_channels=256, out_channels=256,kernel_size=3, padding = 1,stride=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2)
            )
        
        self.conv4 = nn.Sequential(
            nn.Conv2d(in_channels=256, out_channels=512,kernel_size=3, padding = 1,stride=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(in_channels=512, out_channels=512,kernel_size=3, padding = 1,stride=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2)
            )
        self.conv5 = nn.Sequential(
            nn.Conv2d(in_channels=512, out_channels=512,kernel_size=3, padding = 1,stride=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(in_channels=512, out_channels=512,kernel_size=3, padding = 1,stride=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2)
            )
        self.fc_layer = nn.Sequential(
            nn.Linear(512 * 7 * 7, 4096),
            nn.ReLU(inplace=True),
            nn.Dropout(p=0.5),
            nn.Linear(4096, 4096),
            nn.ReLU(inplace=True),
            nn.Dropout(p=0.5),
            nn.Linear(4096, num_classes)
        )

    def forward(self, x):
        x = self.conv1(x)
        x = self.conv2(x)
        x = self.conv3(x)
        x = self.conv4(x)
        x = self.conv5(x)
        x = torch.flatten(x,1)
        x = self.fc_layer(x)

        return x
        

In [ ]:
vgg_model = VGG_11(num_classes = 10)
vgg_model = vgg_model.to(device)

classes =  ('airplance', 'bird', 'car', 'cat', 'deer', 'dog', 'horse', 'monkey', 'ship', 'truck')
import torch.optim as optim

criterion = nn.CrossEntropyLoss().cuda()
optimizer = optim.Adam(vgg_model.parameters(),lr=0.00001)

for epoch in range(10):  # loop over the dataset multiple times
    running_loss = 0.0
    for i, data in tqdm(enumerate(trainloader, 0)):
        # get the inputs
        inputs, labels = data
        inputs, labels = inputs.to(device), labels.to(device)
        # zero the parameter gradients
        optimizer.zero_grad()

        # print(inputs.shape)  
        outputs= vgg_model(inputs)

        # print(outputs.shape)
        # print(labels.shape)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        if i % 50 == 49:    # print every 50 mini-batches
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 50))
            running_loss = 0.0

print('Finished Training')

0it [00:00, ?it/s]

[1,    50] loss: 2.303
[1,   100] loss: 2.303
[1,   150] loss: 2.268


0it [00:00, ?it/s]

[2,    50] loss: 2.104
[2,   100] loss: 2.052
[2,   150] loss: 2.018


0it [00:00, ?it/s]

...

[8,    50] loss: 1.526
[8,   100] loss: 1.549
[8,   150] loss: 1.547


0it [00:00, ?it/s]

[9,    50] loss: 1.504
[9,   100] loss: 1.534
[9,   150] loss: 1.487


0it [00:00, ?it/s]

[10,    50] loss: 1.439
[10,   100] loss: 1.461
[10,   150] loss: 1.516
Finished Training
